[View in Colaboratory](https://colab.research.google.com/github/manan-arya/Analytics_Vidhya_Competition/blob/XGBoost/XGBoost_CV.ipynb)

In [0]:
import pandas as pd
import numpy as np
import xgboost as xgb

In [76]:
!wget https://raw.githubusercontent.com/manan-arya/Analytics_Vidhya_Competition/master/train_NIR5Yl1.csv
!wget https://raw.githubusercontent.com/manan-arya/Analytics_Vidhya_Competition/master/test_8i3B3FC.csv

--2018-09-02 04:47:09--  https://raw.githubusercontent.com/manan-arya/Analytics_Vidhya_Competition/master/train_NIR5Yl1.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 151.101.0.133, 151.101.64.133, 151.101.128.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|151.101.0.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 12564851 (12M) [text/plain]
Saving to: ‘train_NIR5Yl1.csv.3’

train_NIR5Yl1.csv.3 100%[===================>]  11.98M  16.9MB/s    in 0.7s    

2018-09-02 04:47:10 (16.9 MB/s) - ‘train_NIR5Yl1.csv.3’ saved [12564851/12564851]

--2018-09-02 04:47:11--  https://raw.githubusercontent.com/manan-arya/Analytics_Vidhya_Competition/master/test_8i3B3FC.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 151.101.0.133, 151.101.64.133, 151.101.128.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|151.101.0.133|:443... connected.
HTTP request sent, awaiting resp

In [0]:
train=pd.read_csv("train_NIR5Yl1.csv")
test=pd.read_csv("test_8i3B3FC.csv")

In [78]:
train.head()

,ID,Tag,Reputation,Answers,Username,Views,Upvotes
0,52664,a,3942.0,2.0,155623,7855.0,42.0
1,327662,a,26046.0,12.0,21781,55801.0,1175.0
2,468453,c,1358.0,4.0,56177,8067.0,60.0
3,96996,a,264.0,3.0,168793,27064.0,9.0
4,131465,c,4271.0,4.0,112223,13986.0,83.0


In [79]:
test.head()

,ID,Tag,Reputation,Answers,Username,Views
0,366953,a,5645.0,3.0,50652,33200.0
1,71864,c,24511.0,6.0,37685,2730.0
2,141692,i,927.0,1.0,135293,21167.0
3,316833,i,21.0,6.0,166998,18528.0
4,440445,i,4475.0,10.0,53504,57240.0


In [80]:
train = pd.concat([train,pd.get_dummies(train['Tag'], drop_first=True)], axis=1)
train.head()

,ID,Tag,Reputation,Answers,Username,Views,Upvotes,c,h,i,j,o,p,r,s,x
0,52664,a,3942.0,2.0,155623,7855.0,42.0,0,0,0,0,0,0,0,0,0
1,327662,a,26046.0,12.0,21781,55801.0,1175.0,0,0,0,0,0,0,0,0,0
2,468453,c,1358.0,4.0,56177,8067.0,60.0,1,0,0,0,0,0,0,0,0
3,96996,a,264.0,3.0,168793,27064.0,9.0,0,0,0,0,0,0,0,0,0
4,131465,c,4271.0,4.0,112223,13986.0,83.0,1,0,0,0,0,0,0,0,0


In [81]:
test = pd.concat([test,pd.get_dummies(test['Tag'], drop_first=True)], axis=1)
test.head()

,ID,Tag,Reputation,Answers,Username,Views,c,h,i,j,o,p,r,s,x
0,366953,a,5645.0,3.0,50652,33200.0,0,0,0,0,0,0,0,0,0
1,71864,c,24511.0,6.0,37685,2730.0,1,0,0,0,0,0,0,0,0
2,141692,i,927.0,1.0,135293,21167.0,0,0,1,0,0,0,0,0,0
3,316833,i,21.0,6.0,166998,18528.0,0,0,1,0,0,0,0,0,0
4,440445,i,4475.0,10.0,53504,57240.0,0,0,1,0,0,0,0,0,0


In [0]:
train.drop("Tag",axis=1,inplace=True)
test.drop("Tag",axis=1,inplace=True)


In [83]:
test.head()

,ID,Reputation,Answers,Username,Views,c,h,i,j,o,p,r,s,x
0,366953,5645.0,3.0,50652,33200.0,0,0,0,0,0,0,0,0,0
1,71864,24511.0,6.0,37685,2730.0,1,0,0,0,0,0,0,0,0
2,141692,927.0,1.0,135293,21167.0,0,0,1,0,0,0,0,0,0
3,316833,21.0,6.0,166998,18528.0,0,0,1,0,0,0,0,0,0
4,440445,4475.0,10.0,53504,57240.0,0,0,1,0,0,0,0,0,0


In [0]:
X_train=train.drop("Upvotes",axis=1)
y_train=train["Upvotes"]


In [0]:
!export PYTHONPATH=~/xgboost/python-package

In [0]:
X_train_arr = X_train.as_matrix()
test_arr = test.as_matrix()

In [87]:
print("\nSetting up data for XGBoost ...")
# xgboost params
xgb_params = {
    #'eta': 0.037,
    'max_depth': 5,
    'subsample': 0.80,
    'objective': 'reg:linear',
    #'eval_metric': 'mae',
    #'lambda': 0.8,   
    #'alpha': 0.4, 
    #'silent': 1
}

# AttributeError: module 'xgboost' has no attribute 'DMatrix'
dtrain = xgb.DMatrix(X_train_arr, y_train)
dtest = xgb.DMatrix(test_arr)

num_boost_rounds = 20
print("num_boost_rounds="+str(num_boost_rounds))

# train model
print( "\nTraining XGBoost ...")
model = xgb.train(dict(xgb_params, silent=1), dtrain, 
num_boost_round=num_boost_rounds)

print( "\nPredicting with XGBoost ...")
preds = model.predict(dtest)


Setting up data for XGBoost ...
num_boost_rounds=20

Training XGBoost ...

Predicting with XGBoost ...


In [0]:
ID=test['ID']

In [91]:
preds

array([ 46.12435 , 178.4388  ,  27.639656, ...,  75.14111 ,  24.973871,
        27.639656], dtype=float32)

In [0]:
submission = pd.DataFrame({'ID':ID, 'Upvotes':preds})

In [0]:
submission.to_csv('enigma_sub_3.csv', index=False)
from google.colab import files 
files.download('enigma_sub_3.csv')

In [95]:
submission.head()

,ID,Upvotes
0,366953,46.124352
1,71864,178.438797
2,141692,27.639656
3,316833,27.639656
4,440445,87.177223


In [66]:
preds

,Upvotes
0,46.124352
1,178.438797
2,27.639656
3,27.639656
4,87.177223
5,32.273106
6,24.973871
7,75.141113
8,45.453285
9,24.973871
